In [ ]:
import healpy as hp
import mapsims
from glob import glob
import numpy as np
from collections import defaultdict

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
sim = mapsims.from_config(["common.toml", "noise.toml"], override={"channels":"tube:LT6"})

noise = sim.other_components["noise"]

In [ ]:
onthefly = sim.execute()

In [ ]:
hitmaps = {}
ave_hits = {}
for i_tube in range(6, 7):
    tube = f"LT{i_tube}"
    chs = mapsims.parse_channels("tube:"+tube)[0]
    hitmaps[tube], ave_hits[tube] = noise.get_hitmaps(tube)
#    hitmaps[tube] = hp.ud_grade(hitmaps[tube],
#            mapsims.runner.get_default_so_resolution(chs))
    
    for ch, h in zip(chs, hitmaps[tube]):
        h /= h.max()
        #h[h<.01] = 0
        #hp.mollview(h, title=ch.tag)

In [ ]:
all_cl = {}
sky_fractions = defaultdict(dict)
for i_tube in range(6, 7):
    tube = f"LT{i_tube}"
    ellmax = int(1e4)
    m = {}
    cl = []
    folder = "output/noise/0000/"
    for i, ch in enumerate(mapsims.parse_channels("tube:"+tube)[0]):
        filename = glob(folder + f"*{ch.tag}*1_of_1*")[0]
        print("reading " + filename)
        m[i] = hp.read_map(filename, (0,1,2))
        
        npix = len(m[i][0])
        sky_fractions[tube][i]=hp.mask_good(m[i][1]).sum()/npix
        m[i][m[i] == hp.UNSEEN] = 0

        m[i] *= hitmaps[tube][i]
        
        hp.mollview(m[i][1], title=ch.tag)
        nside = hp.npix2nside(npix)
        cl.append(
            np.array(
                hp.anafast(m[i],
                           lmax=min(3*nside-1,ellmax),
                           use_pixel_weights=False)
            ) / np.mean(hitmaps[tube][i]**2) / sky_fractions[tube][i])
    cl = np.array(cl)
    all_cl[tube] = cl
cl = all_cl

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mapsims

In [ ]:
from glob import glob

In [ ]:
import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {'family':'sans-serif',
                  'weight' : 'normal', 'size' : 16}

In [ ]:
import pickle

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2,sharey=True, figsize=(20/3*2, 5*4))
pol = 0
i_tube = 0
for (tube, chs) in noise.tubes.items():
    if tube.startswith("LT6"):
        ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=tube)

        wn = noise.get_white_noise_power(tube, sky_fractions[tube][i])

        m = {}
        for i, ch in enumerate(chs):
            len_cl = len(cl[tube][i])
            ax[i_tube][i].loglog(ell, ps_T[i] * sky_fractions[tube][i], label="Expected T spectrum")
            ax[i_tube][i].loglog(cl[tube][i][pol], label="I", alpha=.5)
            ax[i_tube][i].hlines(wn[i], 100, ell[-1], label="White noise power")

            ax[i_tube][i].set_ylim(1e-8,1e1)
            ax[i_tube][i].set_title(ch, fontsize=16)
            
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
            ax[i_tube][i].legend()
            ax[i_tube][i].grid(True)
        i_tube += 1
        
plt.savefig("plots/LT6_T.png")

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(20/3*2, 5*4))
ylim = (1e-7, 1e0)
i_tube = 0
pol_label = "TQU"
for (tube, chs) in noise.tubes.items():
    ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=tube)
    if tube.startswith("LT6"):

        nside = hp.npix2nside(len(hitmaps[tube][0]))
        pixarea = hp.nside2pixarea(nside)
        wn = noise.get_white_noise_power(tube, sky_fractions[tube][i])
        m = {}
        for i, ch in enumerate(chs):
            ax[i_tube][i].loglog(ell, ps_P[i] * sky_fractions[tube][i], label="Expected P spectrum")
            # noise power is double in polarization
            ax[i_tube][i].hlines(wn[i]*2, 100, ell[-1], label="White noise power")
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
        for pol in [1,2]:
            for i, ch in enumerate(chs):
                len_cl = len(cl[tube][i])
                ax[i_tube][i].loglog(cl[tube][i][pol], label=pol_label[pol], alpha=.5)

                ax[i_tube][i].set_ylim(*ylim)
                ax[i_tube][i].set_title(ch, fontsize=16)
                ax[i_tube][i].legend()
                ax[i_tube][i].grid(True)
        i_tube += 1
    
    

plt.savefig("plots/LT6_P.png")